<center>
    <h1> ILI285 - Computación Científica I  / INF285 - Computación Científica </h1>
    <h2> Tarea 5 </h2>
    <h2> [S]cientific [C]omputing [T]eam </h2>
</center>

_Junio 2017_

In [3]:
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import IntSlider
import numpy as np
import pickle
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output
from ipywidgets import fixed
from IPython.display import display
from scipy.optimize import newton
%matplotlib inline

<div id='obj' />
# 1. Objetivos

<ul>
<li>Usar técnicas de interpolación para poder recorrer imágenes en el tiempo. </li>
<li>Ver como el fenómeno de Runge afecta a la interpolación y observar como los puntos de Chebyshev ayudan para corregir a este ultimo.</li>
</ul>


In [4]:
def Chebyshev_points(xmin,xmax,n):
    ns = np.arange(1,n+1)
    x = np.cos((2*ns-1)*np.pi/(2*n))
    return (xmin+xmax)/2 + (xmax-xmin)*x/2

In [5]:
images_list_faces = [0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80,84,88,92]
t_i_faces = np.array(images_list_faces).astype(float)
c_i_faces = Chebyshev_points(0,92,len(images_list_faces))[::-1]

La interpolación es una herramienta muy útil, la cual nos permite encontrar un polinomio que pasa por puntos en específico y por consecuencia, encontrar un valor numérico para aquellos puntos donde no conocemos su imagen. En esta tarea vamos a hacer uso de esta herramienta para poder observar como una persona va envejeciendo en el tiempo. Para hacer esto, vamos a interpolar fotos de distintos instantes de su vida. 

Primero, se debe construir un polinomio interpolador $P(t)$, el cual evaluado en un instante $\mathit{t}$ de tiempo retornará una matriz (la representación de la foto). Para realizar esta tarea se utilizará la interpolación de Lagrange. Este polinomio tendrá la siguiente estructura.

$$
P(t) = I_{1} \frac{(t-t_{2})(t-t_{3})(t-t_{4})...(t-t_{n})}{(t_{1}-t_{2})(t_{1}-t_{3})(t_{1}-t_{4})...(t_{1}-t_{n})} + I_{2} \frac{(t-t_{1})(t-t_{3})(t-t_{4})...(t-t_{n})}{(t_{2}-t_{1})(t_{2}-t_{3})(t_{2}-t_{4})...(t_{2}-t_{n})} + ... + I_{n} \frac{(t-t_{1})(t-t_{2})(t-t_{3})...(t-t_{n-1})}{(t_{n}-t_{1})(t_{n}-t_{2})(t_{n}-t_{3})...(t_{n}-t_{n-1})}
$$

Aquí $I_{i}$ corresponde a una matriz (la representación de la Foto) y $t_{i}$ el instante de tiempo de esta, $i\in \{1...n\}$. Es fácil ver que este polinomio nos retorna el resultado de una suma de matrices que son _ponderadas_ por los distintos $\mathit{L}_{i}(t)$ de Lagrange.

In [13]:
def lagrange(initial, ts, ys, x):
    """
    Función de Lagrange general, recibe un valor inicial al cual agregar el resultado
    (puede ser cualquier tipo que soporte la opreación __iadd__), los valores a interpolar, las imágenes de estos
    y un valor para evaluar.
    """
    for i, y in enumerate(ys):
        frac_t = np.prod([x - t for k, t in enumerate(ts) if k != i])
        frac_b = np.prod([ts[i] - t for k, t in enumerate(ts) if k != i])
        initial += y * frac_t / frac_b
    return initial

# Pregunta 1 

Implementar una función llamada interpolador que reciba la data a interpolar (las imágenes contenidas en Images.pkl), los índices de las fotos a interpolar, el punto que deseamos evaluar y los puntos de interpolación.  (Puede recibir más parámetros, pero los nombrados son obligatorios). Una vez que la función interpoladora este implementada, utilice su función para completar la variable llamada `I2` que se encuentra en la función adjunta más abajo llamada `show_images`.

In [9]:
def interpolador(ts, ys, x):
    return lagrange(np.zeros(ys.shape[1]), ts, ys, x)

**¿Qué sucede cuando evaluamos en instantes de tiempos que no fueron utilizados para interpolar? ¿Qué pasa con los que sí fueron utilizados?**

La calidad de la imágen generada sin Chebyshev fue muy dependiente de la región que se quería interpolar, tendiendo a tener mejor resultados en el medio de la recta, pero si nos vamos a puntos extremos (no interpolados) tenemos imágenes totalmente alejadas de lo esperado.

Si se evalúa en los puntos que si fueron utilizados obtendremos la imágen original correspondiente a este punto ya que la evaluación de Lagrange eliminará todas las demás imágenes multiplicandolas por $0$.

**¿Hay indicios del fenómeno de Runge?, si es así, ¿Cómo lo identificó?**

Si, es muy claro ya que la imágen en los extremos está totalmente saturada, lo que se debe a evaluaciones que fueron sobre-estimadas por nuestro polinomio.

# Pregunta 2
Utilice su función para completar la variable `I3` que se encuentra en la función adjunta más abajo llamada `show_images`, esta vez deberá interpolar las fotos utilizando los puntos de Chebyshev.

**Utilizando los puntos de Chebyshev, ¿presentan las imágenes obtenidas de la interpolación el fenómeno de Runge?.**

Se elimina en gran medida el fenómeno, siendo ya no notorio. Por lo que conocemos de Chebyshev, es usado para minimizar este fenómeno, por lo que esperaríamos un buen resultado. Sin embargo genera otro problema que veremos en el punto siguiente.

**Explique por qué hay un desfase al momento de evaluar en un instante de tiempo $t_{i}$ entre las imágenes que genera la interpolación hecha en  `I3` con la interpolación en `I2` y las imágenes originales.**

*TODO*

# Pregunta 3

Para poder corregir el desfase que se observa entre las imágenes que genera `I3` con respecto a `I2` y las imágenes originales, es necesario hacer una interpolación paramétrica. Esto último se traduce a interpolar los instantes de tiempo $t_{i}$ respecto a los puntos de Chebyshev, con el fin de que cuando queramos encontrar la foto correspondiente a $t_{i}$, nosotros primero debemos encontrar el punto de Chebyshev $s$ correspondiente a  $t_{i}$ en el polinomio interpolador entre los instantes de tiempo y Chebyshev, luego utilizar este punto de Chebyshev $s$ para evaluar en el polinomio interpolador entre las fotos y Chebyshev para tener devuelta la foto correspondiente a $t_i$. Básicamente generamos la parametrización

$$
<t_i(s),I_i(s)>
$$ 

Implemente una función llamada `interpoladorMejorado` que debe hacer lo siguiente:
1. Interpolar los instantes de tiempo `t_i_faces` utilizando los puntos de Chebyshev `c_i_faces`.
2. Hacer una busqueda de ceros del la función interpoladora anterior, para encontrar el punto de Chebyshev $\mathit{s}$ correspondiente al instante de tiempo $\mathit{t}$.
3. Evaluar el punto de Chebyshev $\mathit{s}$ encontrado en la busqueda de cero anterior, en la función interpoladora de las imagenes con el punto de Chebyshev.
4. Completar la variable `I4` de la función `show_images` utilizando su función `interpoladoraMejorada`. 

**¿Existe desfase? ¿Hay indicios de el fenómeno de Runge? Concluya.**

Al hacer este ajuste de interpolación logramos eliminar el desfase y minimizar el *Fenómeno de Runge*, por lo que obtenemos muy buenos resultados para cada instante de tiempo.

In [7]:
def interpoladorMejorado(ts, ys, x_t):
    return interpolador(c_i_faces, ys, newton(lambda x: x_t - lagrange(0, c_i_faces, ts, x), x_t))

In [8]:
def show_images(x_t=0): 
    with open('images.pkl', 'rb') as handle:
        images = pickle.load(handle) ## Se abre el dataset de caras
    Original = images[x_t] #Aca se extrae la imagen original correspondiente a x_t del dataset de caras.
    I2 = interpolador(t_i_faces,images[images_list_faces],x_t) #Interpolación utilizando los x_t
    I3 = interpolador(c_i_faces,images[images_list_faces],x_t) #Interpolación utilizando los c_i
    I4 = interpoladorMejorado(t_i_faces,images[images_list_faces],x_t) #Interpolación correción
    px,py = 180,139 #Dimension de las imagenes de caras.
    
    f, ax = plt.subplots(1,figsize = (12,3))
    ax.hlines(1,0,93)
    ax.set_xlim(-3,95)
    ax.set_ylim(-3,12)
    ax.plot(images_list_faces,np.ones(np.shape(images_list_faces)),'o',ms = 5, c= 'r', label = "puntos de interpolación")
    ax.plot(x_t,1,'o',ms = 5, c= 'b', label = "puntos de evaluación")
    for t in range(0,len(images_list_faces)):
       ax.text(images_list_faces[t],2, str(images_list_faces[t]), fontsize=10)
    ax.text(x_t,-1, str(x_t), fontsize=15)
    legend = ax.legend(loc='upper center', shadow=True, fontsize='large')
    ax.axis('off')
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex='col', sharey='row', figsize=(16,4))

    ax1.imshow(np.reshape(Original,(px,py)), cmap='gray')
    ax2.imshow(np.reshape(I2,(px,py)), cmap='gray')
    ax3.imshow(np.reshape(I3,(px,py)), cmap='gray')
    ax4.imshow(np.reshape(I4,(px,py)), cmap='gray')
    ax1.set_title('Imagen Original en el instante t = '+ str(x_t))
    ax2.set_title('Interpolación Ptos Equiespaciados')
    ax3.set_title('Interpolación Chebyshev')
    ax4.set_title('Interpolación Chebyshev Mejorada')
    plt.show()
dx = 1
widgets.interact(show_images,x_t = (0,92,dx))

<function __main__.show_images>

# Instrucciones:
 
 
* La estructura del laboratorio es la siguiente:
     1. Título, nombre de estudiante, email y rol.
     2. Introducción.
     3. Desarrollo y análisis de resultados.
     4. Conclusiones.
     5. Referencias.
* La tarea debe ser realizada en `Jupyter` notebook (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, `SymPy`, `Matplotlib` y `ipywidgets`, así como la correcta implementación de algoritmos vectorizados cuando se indique.
* El archivo de entrega debe denominarse Tarea5-rol.tar.gz, y debe contener un directorio con todos los archivos necesarios para ejecutar el notebook, junto con un archivo README indicando explícitamente las librerías o módulos utilizados.
* El descuento por día de atraso será de $30$ puntos, con un máximo de 1 día de atraso. No se recibirán entregas después de este día.
* El trabajo es personal, no se permite compartir código, aunque sí se sugiere discutir aspectos generales con sus compañeros. En caso de sospecha de copia, se citará a los involucrados a defender sus conocimientos de manera presencial, evaluación cuya nota reemplazará la obtenida en la tarea.
* El no seguir estas instrucciones, implica descuentos en su nota obtenida.